In [1]:
#| echo: false
#| label: 024_imports
import os
from math import inf
import numpy as np
import warnings
if not os.path.exists('./figures'):
    os.makedirs('./figures')
warnings.filterwarnings("ignore")

This chapter demonstrates hyperparameter tuning for `river`'s `Hoeffding Adaptive Tree Regressor` with the Friedman drift data set [[SOURCE]](https://riverml.xyz/0.18.0/api/datasets/synth/FriedmanDrift/). The `Hoeffding Adaptive Tree Regressor` is a decision tree that uses the Hoeffding bound to limit the number of splits evaluated at each node. The `Hoeffding Adaptive Tree Regressor` is a regression tree, i.e., it predicts a real value for each sample. The `Hoeffding Adaptive Tree Regressor` is a drift aware model, i.e., it can handle concept drifts.


## The Friedman Drift Data Set

We will use the Friedman synthetic dataset with concept drifts [[SOURCE]](https://riverml.xyz/0.18.0/api/datasets/synth/FriedmanDrift/), see also @frie91a and @ikon11a.
Each observation is composed of ten features. Each feature value is sampled uniformly in [0, 1]. Only the first five features are relevant. The target is defined by different functions depending on the type of the drift. Global Recurring Abrupt drift will be used, i.e., the concept drift appears over the whole instance space.

The target is defined by the following function:
$$
y = 10 \sin(\pi x_0 x_1) + 20 (x_2 - 0.5)^2 + 10 x_3 + 5 x_4 + \epsilon,
$$
where $\epsilon \sim \mathcal{N}(0, 1)$ is normally distributed noise.

We will use the Global Recurring Abrupt drift variant of the Friedman Drift dataset.
There are two points of concept drift, namely $p_1$ and $p_2$.
At the second point, the concept changes to:
$$
y = 10 \sin(\pi x_3 x_5) + 20 (x_1 - 0.5)^2 + 10 x_0 + 5 x_2 + \epsilon,
$$
At the second point of drift the old concept reoccurs.
This can be implemented as follows, see [https://riverml.xyz/latest/api/datasets/synth/FriedmanDrift/](https://riverml.xyz/latest/api/datasets/synth/FriedmanDrift/):

```python
def __iter__(self):
    rng = random.Random(self.seed)

    i = 0
    while True:
        x = {i: rng.uniform(a=0, b=1) for i in range(10)}
        y = self._global_recurring_abrupt_gen(x, i) + rng.gauss(mu=0, sigma=1)

        yield x, y
        i += 1
```


```python
def _global_recurring_abrupt_gen(self, x, index: int):
    if index < self._change_point1 or index >= self._change_point2:
        # The initial concept is recurring
        return (
            10 * math.sin(math.pi * x[0] * x[1]) + 20 * (x[2] - 0.5) ** 2 + 10 * x[3] + 5 * x[4]
        )
    else:
        # Drift: the positions of the features are swapped
        return (
            10 * math.sin(math.pi * x[3] * x[5]) + 20 * (x[1] - 0.5) ** 2 + 10 * x[0] + 5 * x[2]
        )
```

Plotting the first 1000 samples of the Friedman Drift dataset, we can not see the concept drifts at $p_1$ and $p_2$.
Drift can be visualized by plotting the target values over time for constant features, e,g, if $x_0$ is set to $1$ and all other features are set to $0$. This is illustrated in the following plot.


In [2]:
import matplotlib.pyplot as plt
import math
import random

class FriedmanDriftDataset:
    def __init__(self, change_point1=50, change_point2=75, seed=None, constant=False):
        self._change_point1 = change_point1
        self._change_point2 = change_point2
        self.seed = seed
        self.index = 0
        self.rng = random.Random(self.seed)
        self.constant = constant

    def __iter__(self):
        return self

    def __next__(self):
        if self.index >= 100:  # Specifying end of generation
            raise StopIteration
        if self.constant:
            # x[0] is set to 1, all others to 0
            x = {0: 1}
            x.update({i: 0 for i in range(1, 10)})  # All x[i] are 0 for i > 0
        else:
            x = {i: self.rng.uniform(a=0, b=1) for i in range(10)}
        y = self._global_recurring_abrupt_gen(x, self.index) + self.rng.gauss(mu=0, sigma=1)
        result = (x, y, self.index)
        self.index += 1
        return result

    def _global_recurring_abrupt_gen(self, x, index):
        if index < self._change_point1 or index >= self._change_point2:
            return (
                10 * math.sin(math.pi * x[0] * x[1]) +
                20 * (x[2] - 0.5) ** 2 +
                10 * x[3] + 5 * x[4]
            )
        else:
            return (
                10 * math.sin(math.pi * x[3] * x[5]) +
                20 * (x[1] - 0.5) ** 2 +
                10 * x[0] + 5 * x[2]
            )

In [3]:
data_generator = FriedmanDriftDataset(seed=42, change_point1=50, change_point2=75)
data = [data for data in data_generator]  # Now this should work

indices = [i for _, _, i in data]
values = {f"x{i}": [] for i in range(5)}
values["y"] = []

for x, y, _ in data:
    for i in range(5):
        values[f"x{i}"].append(x[i])
    values["y"].append(y)

plt.figure(figsize=(10, 6))

for label, series in values.items():
    plt.plot(indices, series, label=label)

plt.xlabel('Index')
plt.ylabel('Value')
plt.title('')
plt.axvline(x=50, color='k', linestyle='--', label='Drift Point 1')
plt.axvline(x=75, color='r', linestyle='--', label='Drift Point 2')
plt.legend()
plt.grid(True)
plt.show()

<Figure size 3000x1800 with 1 Axes>

In [4]:
# Generation of the dataset with constant x-values
data_generator = FriedmanDriftDataset(seed=42, change_point1=50, change_point2=75, constant=True)
data = [next(data_generator) for _ in range(100)]

# Extracting y-values and indices for plotting
y_values = [y for _, y, _ in data]
indices = range(100)

# Plotting the y-values
plt.figure(figsize=(10, 6))
plt.plot(indices, y_values, label="y Value")

plt.xlabel('Index')
plt.ylabel('y Value')
plt.title('')
plt.axvline(x=50, color='k', linestyle='--', label='Drift Point 1')
plt.axvline(x=75, color='r', linestyle='--', label='Drift Point 2')
plt.legend()
plt.grid(True)
plt.show()

<Figure size 3000x1800 with 1 Axes>

The following parameters are used to generate and handle the data set:

* horizon: The prediction horizon in hours.
* n_samples: The number of samples in the data set.
* p_1: The position of the first concept drift.
* p_2: The position of the second concept drift.
* position: The position of the concept drifts.
* n_train: The number of samples used for training.

We will use `spotRiver`'s `convert_to_df` function [[SOURCE]](https://github.com/sequential-parameter-optimization/spotRiver/blob/main/src/spotRiver/utils/data_conversion.py) to convert the `river` data set to a `pandas` data frame.
Then we add column names x1 until x10 to the first 10 columns of the dataframe and the column name y to the last column of the dataframe.


In [5]:
#| label: 024_data_set

from river.datasets import synth
import pandas as pd
import numpy as np
from spotRiver.utils.data_conversion import convert_to_df

seed = 123
shuffle = True
n_train = 6_000
n_test = 4_000
n_samples = n_train + n_test
target_column = "y"

dataset = synth.FriedmanDrift(
   drift_type='gra',
   position=(n_train/4, n_train/2),
   seed=123
)

train = convert_to_df(dataset, n_total=n_train)
train.columns = [f"x{i}" for i in range(1, 11)] + [target_column]


dataset = synth.FriedmanDrift(
   drift_type='gra',
   position=(n_test/4, n_test/2),
   seed=123
)
test = convert_to_df(dataset, n_total=n_test)
test.columns = [f"x{i}" for i in range(1, 11)] + [target_column]

In [6]:
import matplotlib.pyplot as plt

# Assuming train is your prepared DataFrame from the provided code snippet.
indices = range(len(train))
y_values = train[target_column]

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(indices, y_values, label="y Value", color='blue')

# Marking drift points
drift_points = [n_train / 4, n_train / 2]
for dp in drift_points:
    plt.axvline(x=dp, color='red', linestyle='--', label=f'Drift Point at {int(dp)}')

# To avoid duplicate labels in the legend
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlabel('Index')
plt.ylabel('Target Value (y)')
plt.title('Train Data')
plt.grid(True)
plt.show()

<Figure size 3000x1800 with 1 Axes>

In [7]:
import matplotlib.pyplot as plt

indices = range(len(test))
y_values = test[target_column]

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(indices, y_values, label="y Value", color='blue')

# Marking drift points
drift_points = [n_test / 4, n_test / 2]
for dp in drift_points:
    plt.axvline(x=dp, color='red', linestyle='--', label=f'Drift Point at {int(dp)}')

# To avoid duplicate labels in the legend
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlabel('Index')
plt.ylabel('Target Value (y)')
plt.title('Train Data')
plt.grid(True)
plt.show()

<Figure size 3000x1800 with 1 Axes>

## Setup

Before we consider the detailed experimental setup, we select the parameters that affect run time, initial design size, size of the data set, and the experiment name.

* `MAX_TIME`: The maximum run time in seconds for the hyperparameter tuning process.
* `INIT_SIZE`: The initial design size for the hyperparameter tuning process.
* `PREFIX`: The prefix for the experiment name.

::: {.callout-caution}
### Caution: Run time and initial design size should be increased for real experiments

* `MAX_TIME` is set to one minute for demonstration purposes. For real experiments, this should be increased to at least 1 hour.
* `INIT_SIZE` is set to 5 for demonstration purposes. For real experiments, this should be increased to at least 10.
:::

The `oml_grace_period` defines the number of observations that are used for the initial training of the model.

The `weight_coeff` defines a multiplier for the results: results are multiplied by (step/n_steps)**weight_coeff, where n_steps is the total number of iterations.
Results from the beginning have a lower weight than results from the end if weight_coeff > 1. If weight_coeff == 0, all results have equal weight. Note, that the `weight_coeff` is only used internally for the tuner and does not affect the results that are used for the evaluation or comparisons.

The `weights` provide a flexible way to define specific requirements, e.g., if the memory is more important than the time, the weight for the memory can be increased.

The `TENSORBOARD_CLEAN` argument is set to `True` to archive the TensorBoard folder if it already exists. This is useful if you want to start a hyperparameter tuning process from scratch.
If you want to continue a hyperparameter tuning process, set `TENSORBOARD_CLEAN` to `False`. Then the TensorBoard folder will not be archived and the old and new TensorBoard files will shown in the TensorBoard dashboard.

## SelectSelect Model (`algorithm`) and `core_model_hyper_dict`

`spotPython` hyperparameter tuning approach uses two components:

1. a model (class) and 
2. an associated hyperparameter dictionary. 

Here, the `river` model class `HoeffdingAdaptiveTreeRegressor` [[SOURCE]](https://riverml.xyz/dev/api/tree/HoeffdingAdaptiveTreeRegressor/) is selected.

The corresponding hyperparameters are loaded from the associated dictionary, which is stored as a JSON file [[SOURCE]](https://github.com/sequential-parameter-optimization/spotRiver/blob/main/src/spotRiver/data/river_hyper_dict.json). The JSON file contains hyperparameter type information, names, and bounds. 

The method `add_core_model_to_fun_control` adds the model and the hyperparameter dictionary to the `fun_control` dictionary.

Alternatively, you can load a local hyper_dict. Simply set `river_hyper_dict.json` as the filename. If `filename`is set to `None`, which is the default, the hyper_dict [[SOURCE]](https://github.com/sequential-parameter-optimization/spotRiver/blob/main/src/spotRiver/data/river_hyper_dict.json) is loaded from the `spotRiver` package.

We use the `StandardScaler` [[SOURCE]](https://riverml.xyz/dev/api/preprocessing/StandardScaler/) from `river` as the preprocessing model. The `StandardScaler` is used to standardize the data set, i.e., it has zero mean and unit variance.

## Objective Function

### The Objective Function {#sec-the-objective-function-13}

The objective function `fun_oml_horizon` [[SOURCE]](https://github.com/sequential-parameter-optimization/spotRiver/blob/main/src/spotRiver/fun/hyperriver.py) is selected next.

## Selection of the Objective (Loss) Function

The `metric_sklearn` is used for the sklearn based evaluation via `eval_oml_horizon` [[SOURCE]](https://github.com/sequential-parameter-optimization/spotRiver/blob/main/src/spotRiver/evaluation/eval_bml.py). Here we use the ` mean_absolute_error` [[SOURCE]](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html) as the objective function.

:::{.callout-note}
#### Note: Additional metrics
`spotRiver` also supports additional metrics. For example, the `metric_river` is used for the river based evaluation via `eval_oml_iter_progressive` [[SOURCE]](https://github.com/sequential-parameter-optimization/spotRiver/blob/main/src/spotRiver/evaluation/eval_oml.py). The `metric_river` is implemented to simulate the behaviour of the "original" `river` metrics.
:::

`spotRiver` provides information about the model' s score (metric), memory, and time.
The hyperparamter tuner requires a single objective.
Therefore, a weighted sum of the metric, memory, and time is computed. The weights are defined in the `weights` array.

:::{.callout-note}
#### Note: Weights
The `weights` provide a flexible way to define specific requirements, e.g., if the memory is more important than the time, the weight for the memory can be increased.
:::

The `oml_grace_period` defines the number of observations that are used for the initial training of the model. The `step` defines the iteration number at which to yield results. This only takes into account the predictions, and not the training steps.  The `weight_coeff` defines a multiplier for the results: results are multiplied by (step/n_steps)**weight_coeff, where n_steps is the total number of iterations. 
Results from the beginning have a lower weight than results from the end if weight_coeff > 1. If weight_coeff == 0, all results have equal weight. Note, that the `weight_coeff` is only used internally for the tuner and does not affect the results that are used for the evaluation or comparisons.


In [8]:
from spotPython.utils.init import fun_control_init, design_control_init, surrogate_control_init, optimizer_control_init
from spotRiver.fun.hyperriver import HyperRiver
from spotRiver.hyperdict.river_hyper_dict import RiverHyperDict

# experiment setup
PREFIX = "024"
fun_evals = inf
max_time = 1
init_size = 10

# evaluation setup
horizon = 7*24
metric_sklearn_name = "mean_absolute_error"
oml_grace_period = horizon
weight_coeff = 1.0
weights = np.array([1, 0.01, 0.01])
fun = HyperRiver().fun_oml_horizon

# data preparation
prep_model_name = "StandardScaler"

# model setup
core_model_name = "tree.HoeffdingAdaptiveTreeRegressor"
# core_model_name = "tree.HoeffdingTreeRegressor"
hyperdict = RiverHyperDict

fun_control = fun_control_init(
    PREFIX=PREFIX,
    TENSORBOARD_CLEAN=True,
    core_model_name=core_model_name,
    fun_evals=fun_evals,
    horizon=horizon,
    hyperdict=hyperdict,
    max_time=max_time,
    metric_sklearn_name=metric_sklearn_name,
    noise=True,
    oml_grace_period=oml_grace_period,
    prep_model_name=prep_model_name,
    seed=seed,
    target_column=target_column,
    test=test,
    train=train,
    weight_coeff=weight_coeff,
    weights=weights,)


design_control = design_control_init(
    init_size=init_size,
)

surrogate_control = surrogate_control_init(
    noise=True,
    n_theta=2,
    min_Lambda=1e-3,
    max_Lambda=10,
)

optimizer_control = optimizer_control_init()

Seed set to 123


Moving TENSORBOARD_PATH: runs/ to TENSORBOARD_PATH_OLD: runs_OLD/runs_2024_06_14_12_17_28
Created spot_tensorboard_path: runs/spot_logs/024_maans14_2024-06-14_12-17-28 for SummaryWriter()


In [9]:
# from spotPython.hyperparameters.values import modify_hyper_parameter_levels
# levels = ["LinearRegression"]
# modify_hyper_parameter_levels(fun_control, "leaf_model", levels)

In [10]:
from spotPython.utils.eda import gen_design_table
from spotPython.spot import spot
print(gen_design_table(fun_control))

Seed set to 123


| name                   | type   | default          |   lower |    upper | transform              |
|------------------------|--------|------------------|---------|----------|------------------------|
| grace_period           | int    | 200              |  10     | 1000     | None                   |
| max_depth              | int    | 20               |   2     |   20     | transform_power_2_int  |
| delta                  | float  | 1e-07            |   1e-08 |    1e-06 | None                   |
| tau                    | float  | 0.05             |   0.01  |    0.1   | None                   |
| leaf_prediction        | factor | mean             |   0     |    2     | None                   |
| leaf_model             | factor | LinearRegression |   0     |    2     | None                   |
| model_selector_decay   | float  | 0.95             |   0.9   |    0.99  | None                   |
| splitter               | factor | EBSTSplitter     |   0     |    2     | None           

### Run the `Spot` Optimizer

The class `Spot` [[SOURCE]](https://github.com/sequential-parameter-optimization/spotPython/blob/main/src/spotPython/spot/spot.py) is the hyperparameter tuning workhorse. It is initialized with the following parameters:

* `fun`: the objective function
* `fun_control`: the dictionary with the control parameters for the objective function
* `design`: the experimental design
* `design_control`: the dictionary with the control parameters for the experimental design
* `surrogate`: the surrogate model
* `surrogate_control`: the dictionary with the control parameters for the surrogate model
* `optimizer`: the optimizer
* `optimizer_control`: the dictionary with the control parameters for the optimizer

:::{.callout-note}
#### Note: Total run time
 The total run time may exceed the specified `max_time`, because the initial design (here: `init_size` = INIT_SIZE as specified above) is always evaluated, even if this takes longer than `max_time`.
:::


In [11]:
spot_tuner = spot.Spot(
    fun=fun,
    fun_control=fun_control,
    design_control=design_control,
    surrogate_control=surrogate_control,
    optimizer_control=optimizer_control,
)
res = spot_tuner.run()

spotPython tuning: 2.271145127818984 [----------] 4.80% 


spotPython tuning: 2.271145127818984 [#---------] 6.81% 


spotPython tuning: 2.271145127818984 [#---------] 10.51% 


spotPython tuning: 2.271145127818984 [#---------] 12.46% 


spotPython tuning: 2.271145127818984 [##--------] 19.05% 


...

spotPython tuning: 2.2606419961336455 [########--] 78.58% 


spotPython tuning: 2.170585555779486 [########--] 81.92% 


spotPython tuning: 2.170585555779486 [#########-] 92.52% 


spotPython tuning: 2.170585555779486 [##########] 96.36% 


spotPython tuning: 2.170585555779486 [##########] 100.00% Done...



## Results

### TensorBoard {#sec-tensorboard-10}

Now we can start TensorBoard in the background with the following command, where `./runs` is the default directory for the TensorBoard log files:



```{raw}
tensorboard --logdir="./runs"
```



:::{.callout-tip}
#### Tip: TENSORBOARD_PATH
The TensorBoard path can be printed with the following command:


In [12]:
#| label: 024_tensorboard_path
from spotPython.utils.init import get_tensorboard_path
get_tensorboard_path(fun_control)

'runs/'

In [13]:
#| label: 024_print_results
spot_tuner.print_results(print_screen=False)

[['grace_period', 369.0],
 ['max_depth', 10.0],
 ['delta', 2.687369912822594e-07],
 ['tau', 0.03965702590085796],
 ['leaf_prediction', 1.0],
 ['leaf_model', 0.0],
 ['model_selector_decay', 0.912608961480265],
 ['splitter', 2.0],
 ['min_samples_split', 5.0],
 ['bootstrap_sampling', 0.0],
 ['drift_window_threshold', 256.0],
 ['switch_significance', 0.010066722690551406],
 ['binary_split', 0.0],
 ['max_size', 714.4837650420958],
 ['memory_estimate_period', 6.0],
 ['stop_mem_management', 0.0],
 ['remove_poor_attrs', 0.0],
 ['merit_preprune', 1.0]]

The tuned hyperparameters can be obtained as a dictionary with the following command:


In [14]:
from spotPython.hyperparameters.values import get_tuned_hyperparameters
get_tuned_hyperparameters(spot_tuner, fun_control)

{'grace_period': 369.0,
 'max_depth': 10.0,
 'delta': 2.687369912822594e-07,
 'tau': 0.03965702590085796,
 'leaf_prediction': 'model',
 'leaf_model': 'LinearRegression',
 'model_selector_decay': 0.912608961480265,
 'splitter': 'QOSplitter',
 'min_samples_split': 5.0,
 'bootstrap_sampling': 0,
 'drift_window_threshold': 256.0,
 'switch_significance': 0.010066722690551406,
 'binary_split': 0,
 'max_size': 714.4837650420958,
 'memory_estimate_period': 6.0,
 'stop_mem_management': 0,
 'remove_poor_attrs': 0,
 'merit_preprune': 1}

The results can be saved and reloaded with the following commands:


In [15]:
from spotPython.utils.file import save_pickle, load_pickle
from spotPython.utils.init import get_experiment_name
experiment_name = get_experiment_name(PREFIX)
SAVE_AND_LOAD = False
if SAVE_AND_LOAD == True:
    save_pickle(spot_tuner, experiment_name)
    spot_tuner = load_pickle(experiment_name)

After the hyperparameter tuning run is finished, the progress of the hyperparameter tuning can be visualized. The black points represent the performace values (score or metric) of  hyperparameter configurations from the initial design, whereas the red points represents the  hyperparameter configurations found by the surrogate model based optimization.


In [16]:
spot_tuner.plot_progress(log_y=True, filename=None)

<Figure size 2700x1800 with 1 Axes>

Results can also be printed in tabular form.


In [17]:
print(gen_design_table(fun_control=fun_control, spot=spot_tuner))

| name                   | type   | default          |   lower |   upper | tuned                 | transform              |   importance | stars   |
|------------------------|--------|------------------|---------|---------|-----------------------|------------------------|--------------|---------|
| grace_period           | int    | 200              |    10.0 |  1000.0 | 369.0                 | None                   |         0.01 |         |
| max_depth              | int    | 20               |     2.0 |    20.0 | 10.0                  | transform_power_2_int  |         2.55 | *       |
| delta                  | float  | 1e-07            |   1e-08 |   1e-06 | 2.687369912822594e-07 | None                   |        62.48 | **      |
| tau                    | float  | 0.05             |    0.01 |     0.1 | 0.03965702590085796   | None                   |         0.00 |         |
| leaf_prediction        | factor | mean             |     0.0 |     2.0 | model                 | None   

A histogram can be used to visualize the most important hyperparameters.


In [18]:
spot_tuner.plot_importance(threshold=0.1)

<Figure size 1650x1050 with 1 Axes>

## Get Default Hyperparameters

The default hyperparameters, which will be used for a comparion with the tuned hyperparameters, can be obtained with the following commands:


In [19]:
from spotPython.hyperparameters.values import get_one_core_model_from_X
from spotPython.hyperparameters.values import get_default_hyperparameters_as_array
X_start = get_default_hyperparameters_as_array(fun_control)
model_default = get_one_core_model_from_X(X_start, fun_control, default=True)
model_default

HoeffdingAdaptiveTreeRegressor (
  grace_period=200
  max_depth=1048576
  delta=1e-07
  tau=0.05
  leaf_prediction="mean"
  leaf_model=LinearRegression (
    optimizer=SGD (
      lr=Constant (
        learning_rate=0.01
      )
    )
    loss=Squared ()
    l2=0.
    l1=0.
    intercept_init=0.
    intercept_lr=Constant (
      learning_rate=0.01
    )
    clip_gradient=1e+12
    initializer=Zeros ()
  )
  model_selector_decay=0.95
  nominal_attributes=None
  splitter=EBSTSplitter ()
  min_samples_split=5
  bootstrap_sampling=0
  drift_window_threshold=300
  drift_detector=ADWIN (
    delta=0.002
    clock=32
    max_buckets=5
    min_window_length=5
    grace_period=10
  )
  switch_significance=0.05
  binary_split=0
  max_size=500.
  memory_estimate_period=1000000
  stop_mem_management=0
  remove_poor_attrs=0
  merit_preprune=1
  seed=None
)

:::{.callout-note}
#### Note: `spotPython` tunes numpy arrays
* `spotPython` tunes numpy arrays, i.e., the hyperparameters are stored in a numpy array.
::::

The model with the default hyperparameters can be trained and evaluated with the following commands:


In [20]:
from spotRiver.evaluation.eval_bml import eval_oml_horizon

df_eval_default, df_true_default = eval_oml_horizon(
                    model=model_default,
                    train=fun_control["train"],
                    test=fun_control["test"],
                    target_column=fun_control["target_column"],
                    horizon=fun_control["horizon"],
                    oml_grace_period=fun_control["oml_grace_period"],
                    metric=fun_control["metric_sklearn"],
                )

The three performance criteria, i.e., score (metric), runtime, and memory consumption, can be visualized with the following commands:


In [21]:
#| label: 024_plot_bml_oml_horizon_metrics_default
from spotRiver.evaluation.eval_bml import plot_bml_oml_horizon_metrics, plot_bml_oml_horizon_predictions
df_labels=["default"]
plot_bml_oml_horizon_metrics(df_eval = [df_eval_default], log_y=False, df_labels=df_labels, metric=fun_control["metric_sklearn"])

<Figure size 3000x1500 with 3 Axes>

### Show Predictions

* Select a subset of the data set for the visualization of the predictions:
    * We use the mean, $m$, of the data set as the center of the visualization.
    * We use 100 data points, i.e., $m \pm 50$ as the visualization window.


In [22]:
#| label: 024_plot_bml_oml_horizon_predictions_default
m = fun_control["test"].shape[0]
a = int(m/2)-50
b = int(m/2)+50
plot_bml_oml_horizon_predictions(df_true = [df_true_default[a:b]], target_column=target_column,  df_labels=df_labels)

<Figure size 3000x1500 with 1 Axes>

## Get SPOT Results

In a similar way, we can obtain the hyperparameters found by `spotPython`.


In [23]:
#| label: 024_get_one_core_model_from_X
from spotPython.hyperparameters.values import get_one_core_model_from_X
X = spot_tuner.to_all_dim(spot_tuner.min_X.reshape(1,-1))
model_spot = get_one_core_model_from_X(X, fun_control)

In [24]:
#| label: 024_eval_om_horizon
df_eval_spot, df_true_spot = eval_oml_horizon(
                    model=model_spot,
                    train=fun_control["train"],
                    test=fun_control["test"],
                    target_column=fun_control["target_column"],
                    horizon=fun_control["horizon"],
                    oml_grace_period=fun_control["oml_grace_period"],
                    metric=fun_control["metric_sklearn"],
                )

In [25]:
#| label: 024_plot_bml_oml_horizon_metrics
df_labels=["default", "spot"]
plot_bml_oml_horizon_metrics(df_eval = [df_eval_default, df_eval_spot], log_y=False, df_labels=df_labels, metric=fun_control["metric_sklearn"])

<Figure size 3000x1500 with 3 Axes>

In [26]:
#| label: 024_plot_bml_oml_horizon_predictions
plot_bml_oml_horizon_predictions(df_true = [df_true_default[a:b], df_true_spot[a:b]], target_column=target_column,  df_labels=df_labels)

<Figure size 3000x1500 with 1 Axes>

In [27]:
#| label: 024_plot_actual_vs_predicted
from spotPython.plot.validation import plot_actual_vs_predicted
plot_actual_vs_predicted(y_test=df_true_default[target_column], y_pred=df_true_default["Prediction"], title="Default")
plot_actual_vs_predicted(y_test=df_true_spot[target_column], y_pred=df_true_spot["Prediction"], title="SPOT")

<Figure size 2400x1200 with 2 Axes>

<Figure size 2400x1200 with 2 Axes>

## Visualize Regression Trees


In [28]:
#| label: 024_model_default_learn_one
dataset_f = dataset.take(n_samples)
print(f"n_samples: {n_samples}")
for x, y in dataset_f:
    model_default.learn_one(x, y)

n_samples: 10000


:::{.callout-caution}
### Caution: Large Trees
* Since the trees are large, the visualization is suppressed by default.
* To visualize the trees, uncomment the following line.
:::


In [29]:
# model_default.draw()

In [30]:
#| label: 024_model_default_summary
model_default.summary

{'n_nodes': 45,
 'n_branches': 22,
 'n_leaves': 23,
 'n_active_leaves': 37,
 'n_inactive_leaves': 0,
 'height': 9,
 'total_observed_weight': 14168.0,
 'n_alternate_trees': 4,
 'n_pruned_alternate_trees': 2,
 'n_switch_alternate_trees': 0}

### Spot Model


In [31]:
#| label: 024_model_spot_learn_one
print(f"n_samples: {n_samples}")
dataset_f = dataset.take(n_samples)
for x, y in dataset_f:
    model_spot.learn_one(x, y)

n_samples: 10000


:::{.callout-caution}
### Caution: Large Trees
* Since the trees are large, the visualization is suppressed by default.
* To visualize the trees, uncomment the following line.
:::


In [32]:
# model_spot.draw()

In [33]:
#| label: 024_model_spot_summary
model_spot.summary

{'n_nodes': 29,
 'n_branches': 14,
 'n_leaves': 15,
 'n_active_leaves': 25,
 'n_inactive_leaves': 0,
 'height': 6,
 'total_observed_weight': 14168.0,
 'n_alternate_trees': 2,
 'n_pruned_alternate_trees': 1,
 'n_switch_alternate_trees': 0}

In [34]:
#| label: 024_compare_two_tree_models
from spotPython.utils.eda import compare_two_tree_models
print(compare_two_tree_models(model_default, model_spot))

| Parameter                |   Default |   Spot |
|--------------------------|-----------|--------|
| n_nodes                  |        45 |     29 |
| n_branches               |        22 |     14 |
| n_leaves                 |        23 |     15 |
| n_active_leaves          |        37 |     25 |
| n_inactive_leaves        |         0 |      0 |
| height                   |         9 |      6 |
| total_observed_weight    |     14168 |  14168 |
| n_alternate_trees        |         4 |      2 |
| n_pruned_alternate_trees |         2 |      1 |
| n_switch_alternate_trees |         0 |      0 |


## Detailed Hyperparameter Plots


In [35]:
#| label: 024_plot_important_hyperparameter_contour
spot_tuner.plot_important_hyperparameter_contour(max_imp=3)

grace_period:  0.006030280026637712
max_depth:  2.545327836268568
delta:  62.48147390519431
tau:  0.0034199041598467718
leaf_prediction:  60.945073337765656
leaf_model:  0.06919266400654939
model_selector_decay:  76.18009430627097
splitter:  100.0
min_samples_split:  10.162934876066554
bootstrap_sampling:  0.04342363404816772
drift_window_threshold:  14.002308072542244
switch_significance:  0.4374629659832072
binary_split:  0.0024929768718882066
max_size:  0.02919694543498391
memory_estimate_period:  5.308699471064371
stop_mem_management:  1.1459434151989298
remove_poor_attrs:  9.11831972420368
merit_preprune:  0.04481027067228103
impo: [['grace_period', 0.006030280026637712], ['max_depth', 2.545327836268568], ['delta', 62.48147390519431], ['tau', 0.0034199041598467718], ['leaf_prediction', 60.945073337765656], ['leaf_model', 0.06919266400654939], ['model_selector_decay', 76.18009430627097], ['splitter', 100.0], ['min_samples_split', 10.162934876066554], ['bootstrap_sampling', 0.043423

<Figure size 2700x1800 with 3 Axes>

<Figure size 2700x1800 with 3 Axes>

<Figure size 2700x1800 with 3 Axes>

## Parallel Coordinates Plots


In [36]:
#| label: 024_parallel_plot
spot_tuner.parallel_plot()